In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
import tensorflow as tf
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)

In [5]:
# 加载数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 数据归一化
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# 标签One-Hot编码
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
plt.figure(figsize=(5,5))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i].reshape(28,28), cmap='gray')
    plt.title(np.argmax(y_train[i]))
    plt.axis('off')
plt.show()

In [ ]:
# 初始化模型
model = Sequential()

# 添加卷积层
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))

# 添加池化层
model.add(MaxPooling2D(pool_size=(2, 2)))

# 展平层
model.add(Flatten())

# 全连接层
model.add(Dense(128, activation='relu'))

# 输出层
model.add(Dense(10, activation='softmax'))

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
with tf.device('/CPU:0'):  
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)


In [ ]:
with tf.device('/GPU:0'):  
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)


In [ ]:
# 绘制准确率曲线
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

# 绘制损失曲线
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

In [9]:
# 初始化模型
lenet5 = Sequential()

# 第一个卷积层和池化层
lenet5.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(28,28,1), padding='same'))
lenet5.add(MaxPooling2D(pool_size=(2, 2)))

# 第二个卷积层和池化层
lenet5.add(Conv2D(16, kernel_size=(5, 5), activation='tanh'))
lenet5.add(MaxPooling2D(pool_size=(2, 2)))

# 展平层
lenet5.add(Flatten())

# 全连接层
lenet5.add(Dense(120, activation='tanh'))
lenet5.add(Dense(84, activation='tanh'))

# 输出层
lenet5.add(Dense(10, activation='softmax'))

In [10]:
# 使用SGD优化器
optimizer = SGD(learning_rate=0.01)
lenet5.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history_lenet5 = lenet5.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)


In [ ]:
# 绘制准确率曲线
plt.plot(history_lenet5.history['accuracy'], label='LeNet-5_train_accuracy')
plt.plot(history_lenet5.history['val_accuracy'], label='LeNet-5_val_accuracy')
plt.legend()
plt.show()


In [ ]:
loss, accuracy = lenet5.evaluate(X_test, y_test)
print(f'LeNet-5 Test Loss: {loss}')
print(f'LeNet-5 Test Accuracy: {accuracy}')
